In [ ]:
import pymongo
from pymongo import MongoClient
import pprint
from IPython.display import clear_output

# Replace XXXX with your connection URI from the Atlas UI
client = MongoClient(XXXX)

# Like the last handout, this pipeline will not work on Atlas until MongoDB 3.6 has been released
# If you're testing this before 3.6 is released you can download and install MongoDB 3.5.X locally
# In that case you should use "mongodb://localhost:27107" as your connection URI
pipeline = [
    {
        '$limit': 100
    },
    #here what we're going to do is split on that dot that we see in those values. And then we're going to use an operator 
    #we've not seen yet called arrayElemAt. ArrayElemAt will select the value stored at a particular index within an array. 
    #And here, what we're doing is as the value to arrayElemAt, we're specifying that we're going to use as our array, 
    #the array that we get by splitting on the dot character for the value of lastUpdated
    {
        '$addFields': {
            'lastupdated': {
                '$arrayElemAt': [
                    {'$split': ["$lastupdated", "."]},
                    0
                ]}
        }
    },
    {
        '$project': {
            'title': 1,
            'year': 1,
            'directors': {'$split': ["$director", ", "]},
            'actors': {'$split': ["$cast", ", "]},
            'writers': {'$split': ["$writer", ", "]},
            'genres': {'$split': ["$genre", ", "]},
            'languages': {'$split': ["$language", ", "]},
            'countries': {'$split': ["$country", ", "]},
            'plot': 1,
            'fullPlot': "$fullplot",
            'rated': "$rating",
            'released': {
                '$cond': {
                    'if': {'$ne': ["$released", ""]},
                    'then': {
                        '$dateFromString': {
                            'dateString': "$released"
                        }
                    },
                    'else': ""}},
            'runtime': 1,
            'poster': 1,
            'imdb': {
                'id': "$imdbID",
                'rating': "$imdbRating",
                'votes': "$imdbVotes"
                },
            'metacritic': 1,
            'awards': 1,
            'type': 1,
            #So continuing with the reshaping that we're doing here, as we did with released, 
            #let's parse a time stamp out of the last updated field
            'lastUpdated': {
                '$cond': {
                    'if': {'$ne': ["$lastupdated", ""]},
                    'then': {
                        '$dateFromString': {
                            'dateString': "$lastupdated",
                            'timezone': "America/New_York"
                        }
                    },
                    'else': ""}}
        }
    },
    {
        '$out': "movies_scratch"
    }
]

clear_output()
pprint.pprint(list(client.mflix.movies_initial.aggregate(pipeline)))